In [ ]:
import pandas as pd

V1_PATH = "vuelta2.xlsx" 
V2_PATH = "votacion_presupuesto.xlsx"
OUTPUT_EXCEL = "analisis_votaciones_presupuesto.xlsx"

SHEET_V1 = "vuelta2"  
SHEET_V2 = "presupuesto"   

# ====== FUNCIONES DE CARGA Y LIMPIEZA ======

def cargar_votacion(path, nombre_ronda, sheet_name=None):
    """
    Lee un Excel de votación y lo deja con columnas estandarizadas:
    nombre, bloque, voto, ronda
    """
    df = pd.read_excel(path, sheet_name=sheet_name)

    # Normalizar nombres de columnas
    df.columns = [c.strip().upper() for c in df.columns]

    # Tratar de mapear a las columnas que nos interesan
    col_nombre = None
    col_bloque = None
    col_voto = None

    for c in df.columns:
        if "NOMBRE" in c:
            col_nombre = c
        if "BLOQUE" in c:
            col_bloque = c
        if "VOTO" in c or "VOTO EMITIDO" in c:
            col_voto = c

    if not col_nombre or not col_voto:
        raise ValueError(
            f"No encontré columnas de NOMBRE/VOTO en {path}. "
            f"Columnas encontradas: {df.columns.tolist()}"
        )

    # Si no trae bloque, lo dejamos como NaN
    if col_bloque is None:
        df["BLOQUE"] = None
        col_bloque = "BLOQUE"

    out = pd.DataFrame()
    out["nombre"] = df[col_nombre].astype(str).str.strip()
    out["bloque"] = df[col_bloque].astype(str).str.strip()
    out["voto"]   = df[col_voto].astype(str).str.strip().str.upper()
    out["ronda"]  = nombre_ronda

    return out


# ====== CARGAR LAS DOS RONDAS ======

v1 = cargar_votacion(V1_PATH, "primera", sheet_name=SHEET_V1)
v2 = cargar_votacion(V2_PATH, "segunda", sheet_name=SHEET_V2)

# Unir por nombre (inner join: solo quienes aparecen en ambas rondas)
merged = (
    v1.merge(
        v2,
        on="nombre",
        suffixes=("_1", "_2"),  # _1 = primera vuelta, _2 = segunda
        how="inner",
    )
)

# Opcional: ordenar
merged = merged.sort_values("nombre").reset_index(drop=True)

# ====== ESTADOS CANÓNICOS (para la matriz tipo Markov) ======
ESTADOS = ["A FAVOR", "EN CONTRA", "AUSENTE", "LICENCIA"]

def normalizar_estado(s):
    s = str(s).strip().upper()
    s = s.replace("Á", "A")
    for e in ESTADOS:
        if s == e:
            return e
    return s

merged["voto_1"] = merged["voto_1"].map(normalizar_estado)
merged["voto_2"] = merged["voto_2"].map(normalizar_estado)

# =========  CATEGORÍA DE CAMBIO POR DIPUTADO  =========

def clasificar_cambio(row):
    v1 = row["voto_1"]
    v2 = row["voto_2"]

    # 1) Se mantiene igual
    if v1 == v2:
        return "Se mantiene"

    # 2) Cambia opinión entre A FAVOR y EN CONTRA
    if (v1 == "A FAVOR" and v2 == "EN CONTRA") or (v1 == "EN CONTRA" and v2 == "A FAVOR"):
        return "Cambia opinion Favor/Contra"

    # 3) Se activa: no votaba -> vota
    if v1 in ["AUSENTE", "LICENCIA"] and v2 in ["A FAVOR", "EN CONTRA"]:
        return "Se activa (no votaba -> vota)"

    # 4) Se desactiva: votaba -> no vota
    if v1 in ["A FAVOR", "EN CONTRA"] and v2 in ["AUSENTE", "LICENCIA"]:
        return "Se desactiva (vota -> no vota)"

    # 5) Cambia tipo de no voto
    if v1 in ["AUSENTE", "LICENCIA"] and v2 in ["AUSENTE", "LICENCIA"]:
        return "Cambia tipo de no voto"

    # 6) Otros cambios raros (por si hay textos distintos)
    return "Otro cambio"

merged["categoria_cambio"] = merged.apply(clasificar_cambio, axis=1)

# =========  CONJUNTOS ESPECÍFICOS QUE YA TENÍAS  =========

# 1) EN CONTRA en la primera y A FAVOR en la segunda
contra_a_favor = merged[
    (merged["voto_1"] == "EN CONTRA") & (merged["voto_2"] == "A FAVOR")
].copy()

# 2) AUSENTE o LICENCIA en la primera y SÍ votaron en la segunda
aus_lic_1_y_votan_2 = merged[
    (merged["voto_1"].isin(["AUSENTE", "LICENCIA"])) &
    (merged["voto_2"].isin(["A FAVOR", "EN CONTRA"]))
].copy()

# 3) A FAVOR en la primera y cambian de opinión en la segunda (a cualquier otro estado)
favor_1_cambian_2 = merged[
    (merged["voto_1"] == "A FAVOR") & (merged["voto_2"] != "A FAVOR")
].copy()

# =========  NUEVOS SUBCONJUNTOS ÚTILES =========

# Se mantienen igual (cualquier estado)
se_mantienen = merged[merged["voto_1"] == merged["voto_2"]].copy()

# Cambian de opinión solo Favor <-> Contra
cambian_opinion_favor_contra = merged[
    ((merged["voto_1"] == "A FAVOR") & (merged["voto_2"] == "EN CONTRA")) |
    ((merged["voto_1"] == "EN CONTRA") & (merged["voto_2"] == "A FAVOR"))
].copy()

# Se activan (no votaban -> votan)  [alias de aus_lic_1_y_votan_2]
se_activan = aus_lic_1_y_votan_2.copy()

# Se desactivan (votaban -> no votan)
se_desactivan = merged[
    (merged["voto_1"].isin(["A FAVOR", "EN CONTRA"])) &
    (merged["voto_2"].isin(["AUSENTE", "LICENCIA"]))
].copy()

# Cambian tipo de no voto (AUSENTE <-> LICENCIA)
cambian_tipo_no_voto = merged[
    ((merged["voto_1"] == "AUSENTE") & (merged["voto_2"] == "LICENCIA")) |
    ((merged["voto_1"] == "LICENCIA") & (merged["voto_2"] == "AUSENTE"))
].copy()

# =========  MATRIZ DE TRANSICIÓN =========

transition_counts = (
    merged
    .groupby(["voto_1", "voto_2"])
    .size()
    .unstack(fill_value=0)
    .reindex(index=ESTADOS, columns=ESTADOS, fill_value=0)
)

transition_probs = transition_counts.div(
    transition_counts.sum(axis=1).replace(0, pd.NA), axis=0
)

# Todas las transiciones con conteo (lista larga)
resumen_transiciones = (
    merged
    .groupby(["voto_1", "voto_2"])
    .size()
    .reset_index(name="conteo")
    .sort_values("conteo", ascending=False)
)

# =========  Transiciones por bloque =========

transiciones_por_bloque = (
    merged
    .groupby(["bloque_1", "voto_1", "voto_2"])
    .size()
    .reset_index(name="conteo")
)

# =========  EXPORTAR A EXCEL =========

with pd.ExcelWriter(OUTPUT_EXCEL, engine="xlsxwriter") as writer:
    merged.to_excel(writer, sheet_name="Votos_unidos", index=False)
    contra_a_favor.to_excel(writer, sheet_name="Contra_a_Favor", index=False)
    aus_lic_1_y_votan_2.to_excel(writer, sheet_name="AusLic_a_Votan", index=False)
    favor_1_cambian_2.to_excel(writer, sheet_name="Favor_cambia", index=False)

    se_mantienen.to_excel(writer, sheet_name="Se_mantienen", index=False)
    cambian_opinion_favor_contra.to_excel(writer, sheet_name="Cambian_Fav_Contra", index=False)
    se_activan.to_excel(writer, sheet_name="Se_activan", index=False)
    se_desactivan.to_excel(writer, sheet_name="Se_desactivan", index=False)
    cambian_tipo_no_voto.to_excel(writer, sheet_name="Cambian_tipo_no_voto", index=False)

    transition_counts.to_excel(writer, sheet_name="Matriz_conteos")
    transition_probs.to_excel(writer, sheet_name="Matriz_probabilidades")
    resumen_transiciones.to_excel(writer, sheet_name="Transiciones_todas", index=False)

    transiciones_por_bloque.to_excel(writer, sheet_name="Trans_por_bloque", index=False)

print("Listo. Resultados guardados en:", OUTPUT_EXCEL)
print("Total diputados emparejados:", len(merged))
print("EN CONTRA -> A FAVOR:", len(contra_a_favor))
print("AUSENTE/LICENCIA -> Votan:", len(aus_lic_1_y_votan_2))
print("A FAVOR en 1ra y cambian:", len(favor_1_cambian_2))


Listo. Resultados guardados en: analisis_votaciones.xlsx
Total diputados emparejados: 160
EN CONTRA -> A FAVOR: 9
AUSENTE/LICENCIA -> Votan: 3
A FAVOR en 1ra y cambian: 12
